<a href="https://colab.research.google.com/github/ADWAITsv1/Assignment/blob/main/Database_of_the_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd
# List to store scraped product information
def scrape_product_info(url, headers):
    page_number = 1
    product_info = []
    while True:
        current_url = f'{url}&page={page_number}'
        page = requests.get(current_url, headers=headers)
        soup = BeautifulSoup(page.content, "html.parser")

        name_elements = soup.select('span.a-size-base-plus.a-color-base.a-text-normal')
        price_elements = soup.select('span.a-price-whole')

        if not name_elements:
            break

        for name_element, price_element in zip(name_elements, price_elements):
            product_info.append({
                'Product Name': name_element.get_text(strip=True),
                'Price': price_element.get_text(strip=True)
            })
# Add a delay of 10 seconds
        page_number += 1
        time.sleep(10)

    return product_info

# URL and headers as in your existing code
URL = 'https://www.amazon.co.jp/s?k=badminton+racket&crid=2BY744N2G634K&sprefix=badminton+racket%2Caps%2C223&ref=nb_sb_noss_1'
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# scrape product information from all pages
scraped_product_info = scrape_product_info(URL, headers)

# Print the scraped product information
for product in scraped_product_info:
    print(f"Product Name: {product['Product Name']}, Price: {product['Price']}")

# Write the scraped product information to a CSV file
output_file_path = 'scraped_product_info.csv'
with open(output_file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Product Name', 'Price'])  # Write header
    for product in scraped_product_info:
        writer.writerow([product['Product Name'], product['Price']])

print(f"Scraped product information has been saved to {output_file_path}")

# Display the first few rows of the CSV file using pandas
df = pd.read_csv(output_file_path)
print(df.head())


Product Name: ヨネックス(YONEX) バドミントン ラケット マッスルパワー2 (ガット張り上げ済) MP2, Price: 2,838
Product Name: YONEX Nanoray 10F Hi-Flex バドミントンラケット 張り済み, Price: 5,643
Product Name: ヨネックスNanoray 6000i g4-uバドミントンラケット, Price: 1,946
Product Name: Senstonバドミントンラケットグラファイトカーボンファイバーシングルハイエンドバドミントンラケット,グリップベルト1つ、ラケットバッグ1つ…, Price: 2,966
Product Name: YONEX グラファイトバドミントンラケット Astrox Liteシリーズ (G4、77g、30ポンドテンション), Price: 3,899
Product Name: 【Amazon.co.jp 限定】Prince(プリンス) バドミントン ラケット SYNERGY LITE 90 (シナジーライト90)カーボン×軽量アルミ グリップサイズ:G5 [張上げ], Price: 7,677
Product Name: ヨネックス(YONEX) バドミントン ラケット アストロクス88Dゲーム (フレームのみ) キャメルゴールド AX88DG, Price: 3,580
Product Name: Wilson(ウイルソン) バドミントンラケット 初心者向け [ガット張り上げ済み] BEAR BADMINTON RACKET (ベア バドミントンラケット), Price: 14,975
Product Name: ヨネックス(YONEX) バドミントン ラケットアイソメトリック TR1 (フレームのみ) ISO-TR1, Price: 1,833
Product Name: ヨネックス(YONEX) バドミントンラケット アストロクス99ゲーム パワー 打ち分け 中級者 ホワイトタイガー(825) AX99G, Price: 7,959
Product Name: バドミントン ラケット B4000G シャトル付き 初心者 練習用 トレーニング 水鳥 スタンダード ヨネックス ケース ガット張り上げ済み, Price: 17,05

In [12]:
import sqlite3

def create_database_and_table(database_path):
    # Establish a connection to the SQLite database
    con = sqlite3.connect(database_path)
    cur = con.cursor()

#CREATE TABLE
    cur.execute('''
        CREATE TABLE IF NOT EXISTS ProductInfo (
            ProductName TEXT,
            Price TEXT
        )
    ''')

# Commit changes and close the connection
    con.commit()
    con.close()

def insert_scraped_data_into_database(scraped_data, database_path):
# Connection to the SQLite database
    con = sqlite3.connect(database_path)
    cur = con.cursor()

# Inserting scraped data into the database
    for product in scraped_data:
        cur.execute('INSERT INTO ProductInfo VALUES (?, ?)', (product['Product Name'], product['Price']))


    con.commit()
    con.close()

# SQLite database path
database_path = 'amazon_data.db'

# Create the database and table
create_database_and_table(database_path)

# Insert scraped data into the database
insert_scraped_data_into_database(scraped_product_info, database_path)


In [15]:
import sqlite3

# SQLite database path
database_path = 'amazon_data.db'

# Establish a connection to the SQLite database
con = sqlite3.connect(database_path)
cur = con.cursor()

# Query all data from the ProductInfo table
cur.execute('SELECT * FROM ProductInfo')
all_data = cur.fetchall()

# Print the result
for row in all_data:
    print(f"Product Name: {row[0]}, Price: {row[1]}")

# Close the connection
con.close()


Product Name: ヨネックス(YONEX) バドミントン ラケット マッスルパワー2 (ガット張り上げ済) MP2, Price: 2,838
Product Name: YONEX Nanoray 10F Hi-Flex バドミントンラケット 張り済み, Price: 5,643
Product Name: ヨネックスNanoray 6000i g4-uバドミントンラケット, Price: 1,946
Product Name: Senstonバドミントンラケットグラファイトカーボンファイバーシングルハイエンドバドミントンラケット,グリップベルト1つ、ラケットバッグ1つ…, Price: 2,966
Product Name: YONEX グラファイトバドミントンラケット Astrox Liteシリーズ (G4、77g、30ポンドテンション), Price: 3,899
Product Name: 【Amazon.co.jp 限定】Prince(プリンス) バドミントン ラケット SYNERGY LITE 90 (シナジーライト90)カーボン×軽量アルミ グリップサイズ:G5 [張上げ], Price: 7,677
Product Name: ヨネックス(YONEX) バドミントン ラケット アストロクス88Dゲーム (フレームのみ) キャメルゴールド AX88DG, Price: 3,580
Product Name: Wilson(ウイルソン) バドミントンラケット 初心者向け [ガット張り上げ済み] BEAR BADMINTON RACKET (ベア バドミントンラケット), Price: 14,975
Product Name: ヨネックス(YONEX) バドミントン ラケットアイソメトリック TR1 (フレームのみ) ISO-TR1, Price: 1,833
Product Name: ヨネックス(YONEX) バドミントンラケット アストロクス99ゲーム パワー 打ち分け 中級者 ホワイトタイガー(825) AX99G, Price: 7,959
Product Name: バドミントン ラケット B4000G シャトル付き 初心者 練習用 トレーニング 水鳥 スタンダード ヨネックス ケース ガット張り上げ済み, Price: 17,05